In [2]:
import numpy as np
import scipy.fft as fft
import matplotlib.pyplot as plt

In [ ]:
# Setting up the signal
F_s =1e6
T = 1/F_s
f_0 = 1e5
omega_0 = 2*np.pi*f_0
phi = np.pi/8
A = 1

N = 513
n_0 = -256
t = np.arange(n_0, n_0+N)*T

SNR_db = np.arange(-10, 60, 10)
SNR_num = 10**(SNR_db/10)

k = np.arange(10,20,2)
M = 2**k

for i, SNR in enumerate(SNR_num):
    for j, m in enumerate(M):
        temp_omega = np.zeros(1000)
        temp_phi = np.zeros(1000)
        for k in range(1000):
            noise_real = np.random.normal(0, A**2/(2*SNR), N)
            noise_complex = np.random.normal(0, A**2/(2*SNR), N)*1j
            signal = A*np.exp(1j*(omega_0*t+phi)) + noise_real+ noise_complex
            F = fft.fft(signal, n=m, norm='ortho')
            m_star = np.argmax(np.abs(F))
            omega_hat = 2*np.pi*m_star/(m*T)
            phi_hat = np.angle(np.exp(-1j*omega_hat*n_0*T)*F[m_star])
            temp_omega[k] = np.abs(omega_hat-omega_0)
            temp_phi[k] = np.abs(phi_hat-phi)
        print('SNR: ', SNR_db[i], 'M: ', m, 'omega: ', np.var(temp_omega, ddof=1), 'phi: ', np.var(temp_phi, ddof=1))
        if np.var(temp_omega, ddof=1) == 0:
            print(np.max(np.abs(temp_omega))-temp_omega[0])
        

SNR:  -10 M:  1024 omega:  1777046179458.64 phi:  0.5601781731598446
SNR:  -10 M:  4096 omega:  1641245676178.2317 phi:  0.5522122095287076
SNR:  -10 M:  16384 omega:  1721685875608.9497 phi:  0.5519867557378337
SNR:  -10 M:  65536 omega:  1687567716255.796 phi:  0.515260172600672
SNR:  -10 M:  262144 omega:  1780051308688.8313 phi:  0.5308315043166598
SNR:  0 M:  1024 omega:  0.0 phi:  0.00019208009077354438
0.0
SNR:  0 M:  4096 omega:  11546.522422658452 phi:  0.0001704184271322854
SNR:  0 M:  16384 omega:  2318.701562326617 phi:  0.00017462609918638373
SNR:  0 M:  65536 omega:  7579.136229431301 phi:  0.00017912281079047709
SNR:  0 M:  262144 omega:  8140.024217959603 phi:  0.0001779826476387898
SNR:  10 M:  1024 omega:  0.0 phi:  1.934476211979624e-06
0.0
SNR:  10 M:  4096 omega:  0.0 phi:  1.8640218727743893e-06
0.0
SNR:  10 M:  16384 omega:  29.295940594601316 phi:  1.6439551790642282e-06
SNR:  10 M:  65536 omega:  66.35898583990894 phi:  1.90774746555216e-06
SNR:  10 M:  262144 